In [43]:
def importdata():
    
    df = pd.read_csv('/Users/jonweske/devment/BA_nextbike/data/df_points_valid_geo_v01.csv')

    df = df[['bike_id','lat','lng','timestood','time_start','time_end']]
    
    #df['date_start'] = df['time_start']
    #df['date_end'] = df['time_end']
    #df['daytime_start'] = df['time_start']
    #df['daytime_end'] = df['time_end']
    
    df['flag'] = 0
    
    
    
    return df

# Main

In [44]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm
import pickle
from datetime import timedelta, datetime, date


df = importdata()


#df


In [45]:
df['timestood'] = pd.to_timedelta(df['timestood'],unit='s')
df['time_start'] = pd.to_datetime(df['time_start'],unit='s')
df['time_end'] = pd.to_datetime(df['time_end'],unit='s')


df

,bike_id,lat,lng,timestood,time_start,time_end,flag
0,71949.0,51.334071,12.399960,0 days 01:27:18,2018-12-31 23:14:49,2019-01-01 00:42:07,0
1,71949.0,51.335429,12.324583,0 days 02:02:31,2019-01-01 01:41:33,2019-01-01 03:44:04,0
2,71949.0,51.343813,12.381495,0 days 09:52:19,2019-01-01 04:14:54,2019-01-01 14:07:13,0
3,71949.0,51.341518,12.365614,2 days 07:15:12,2019-01-01 14:13:45,2019-01-03 21:28:57,0
4,71949.0,51.345950,12.356816,0 days 12:40:42,2019-01-03 21:35:13,2019-01-04 10:15:55,0
...,...,...,...,...,...,...,...
550665,75870.0,51.329036,12.339146,0 days 00:26:20,2019-12-29 14:42:51,2019-12-29 15:09:11,0
550666,75870.0,51.320889,12.343077,0 days 16:59:12,2019-12-29 15:13:29,2019-12-30 08:12:41,0
550667,75870.0,51.332209,12.366046,0 days 08:35:54,2019-12-30 08:23:51,2019-12-30 16:59:45,0
550668,105465.0,0.000000,0.000000,0 days 17:53:59,2019-12-09 16:49:55,2019-12-10 10:43:54,0


In [46]:
for ind in tqdm(df.index):
    ts = df['time_start'][ind]
    te = df['time_end'][ind]
    t0 = ts.replace(hour=23, minute=59, second=59)
    if ts + df['timestood'][ind] > t0:
        df['flag'][ind] = 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 550670/550670 [00:50<00:00, 10834.22it/s]


In [47]:
from tqdm import tqdm

changed = 1

while changed:
    changed = 0
    for ind in tqdm(df.index):
        ts = df['time_start'][ind]
        te = df['time_end'][ind]
        t0 = ts.replace(hour=23, minute=59, second=59)
        
        # is timestood greater than same date 23:59:59
        #   -> over midnight
        if ts + df['timestood'][ind] > t0:
            changed = 1
            
            underhang = t0 - ts
            overhang = te - t0
            
            # fix current day
            df['time_end'][ind] = ts.replace(hour=23, minute=59, second=59)
            df['timestood'][ind] = underhang
            
            # add new row
            time_start = ts.replace(hour=0, minute=0, second=0)
            time_start += timedelta(days=1)
            
            flag = 0
            if time_start + overhang > time_start.replace(hour=23, minute=59, second=59):
                flag = 1
            
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['lng'][ind], 'lat': df['lat'][ind],
                    'timestood': overhang2, 'time_start': time_start2, 'time_end': te2, 'flag' : flag}
            df = df.append(dict, ignore_index=True)

 50%|██████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                              | 275528/550670 [28:20<28:17, 162.05it/s]


KeyboardInterrupt: 

In [ ]:
df

In [28]:
df.to_csv("../data/df_points_valid_geo_split_at_midnight_v03.csv",index=False)

In [42]:
df['timestood'].max()

Timedelta('249 days 12:01:41')

0 Loop
550670

1 Loop
667360 209 days 01:14:44

2 Loop
683917 208 days 01:14:44

3 Loop
690781 207 days 01:14:44

4 Loop
694443 206 days 01:14:44